<a href="https://colab.research.google.com/github/HamzahDrawsheh/Analyzing-Customer-Churn-in-a-Telecom-Company/blob/main/Image's_novelty_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CELL 1 — Install Dependencies**

In [ ]:
!pip install -q diffusers transformers accelerate torch torchvision
!pip install -q datasets matplotlib scikit-learn

# **CELL 2 — Imports & Setup**

In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from diffusers import DiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import pandas as pd
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)

print("Device:", device)

# **Load SDXL Model**

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True
)
pipe.to(device)

pipe.enable_attention_slicing()

# **Define Prompts**

In [ ]:
prompts = {
    "level1": [
        "a dog playing in a park",
        "a red car on a highway",
        "a cup of coffee on a table",
        "a person reading a book",
        "a mountain under blue sky"
    ],
    "level2": [
        "a transparent elephant made of glass",
        "a fish flying over a desert city",
        "a library floating in the ocean",
        "a robot made of tree branches",
        "a burning iceberg in the night"
    ],
    "level3": [
        "a square sun melting into gravity",
        "a whispering shadow made of mathematics",
        "liquid time sleeping on a staircase",
        "a silent explosion of colorless fire",
        "invisible architecture dreaming"
    ]
}

# **Generate Images**

In [ ]:
images_data = []

for level, prompt_list in prompts.items():
    for prompt in prompt_list:
        for i in range(4):
            generator = torch.Generator(device).manual_seed(seed + i)

            image = pipe(
                prompt,
                num_inference_steps=30,
                guidance_scale=7.5,
                generator=generator
            ).images[0]

            filename = f"{level}_{prompt[:20].replace(' ','_')}_{i}.png"
            path = os.path.join(output_dir, filename)
            image.save(path)

            images_data.append({
                "level": level,
                "prompt": prompt,
                "path": path
            })

print("Total images:", len(images_data))

# **Load CLIP Model**

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# **Compute CLIP Alignment**

# **Compute Intra-Prompt Diversity**

In [ ]:
def compute_clip_score(image_path, text):
    image = Image.open(image_path).convert("RGB")

    image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
    text_inputs = clip_processor(text=[text], return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        vision_outputs = clip_model.vision_model(pixel_values=image_inputs["pixel_values"])
        image_emb = vision_outputs.pooler_output

        text_outputs = clip_model.text_model(
            input_ids=text_inputs["input_ids"],
            attention_mask=text_inputs["attention_mask"]
        )
        text_emb = text_outputs.pooler_output

    image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)
    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)

    return (image_emb @ text_emb.T).item()

# **Approx Novelty vs CIFAR-10 dataset**

In [ ]:
def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")

    inputs = clip_processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        vision_outputs = clip_model.vision_model(pixel_values=inputs["pixel_values"])
        emb = vision_outputs.pooler_output

    emb = emb / emb.norm(dim=-1, keepdim=True)

    return emb.squeeze(0).cpu().numpy()

In [ ]:
baseline_embs = []

for i in tqdm(range(500)):
    image, _ = cifar[i]
    image = image.convert("RGB")

    inputs = clip_processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        vision_outputs = clip_model.vision_model(pixel_values=inputs["pixel_values"])
        emb = vision_outputs.pooler_output

    emb = emb / emb.norm(dim=-1, keepdim=True)

    baseline_embs.append(emb.squeeze(0).cpu().numpy())

baseline_embs = np.vstack(baseline_embs)

print("Baseline shape:", baseline_embs.shape)

In [ ]:
for item in tqdm(images_data):
    emb = get_image_embedding(item["path"]).reshape(1, -1)
    sim = cosine_similarity(emb, baseline_embs)
    item["approx_novelty"] = 1 - np.mean(sim)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

diversity_per_level = {}

df = pd.DataFrame(images_data)

for level in df['level'].unique():
    group = df[df['level'] == level]
    embeddings = np.vstack([get_image_embedding(p) for p in group['path']])
    sim_matrix = cosine_similarity(embeddings)

    diversity = 1 - np.mean(sim_matrix)
    diversity_per_level[level] = diversity


df['diversity'] = df['level'].map(diversity_per_level)

# **Results Table**

In [ ]:
df = pd.DataFrame(images_data)
df['diversity'] = df['level'].map(diversity_per_level)

summary = df.groupby("level").agg({
    "clip_score": "mean",
    "diversity": "mean",
    "approx_novelty": "mean"
})

print(summary)

# **Plot Results**

In [ ]:
plt.figure()
plt.plot(summary.index, summary["clip_score"])
plt.title("CLIP Alignment by Level")
plt.show()

plt.figure()
plt.plot(summary.index, summary["diversity"])
plt.title("Diversity by Level")
plt.show()

plt.figure()
plt.plot(summary.index, summary["approx_novelty"])
plt.title("Approx Novelty by Level")
plt.show()